In [1]:
#import required libraries
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score, matthews_corrcoef, precision_score, f1_score, recall_score
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, SimpleRNN,Dropout
from tensorflow.keras.utils import to_categorical
import pandas as pd
import time
import psutil
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras

tf.keras.utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()

2024-04-20 17:52:04.353126: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-20 17:52:04.353195: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-20 17:52:04.356009: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-20 17:52:04.377351: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-20 17:52:06.960414: W ten

In [2]:
#!pip install tensorflow-tensorrt


In [3]:
#!pip install opencv-python
#!pip install imutils
#!pip install scikit-learn
#!pip install xgboost


In [4]:
#Class to load the dataset images from device

class SimpleDatasetLoader:
    # Method: Constructor
    def __init__(self, preprocessors=None):
        """
        :param preprocessors: List of image preprocessors
        """
        self.preprocessors = preprocessors

        if self.preprocessors is None:
            self.preprocessors = []

    # Method: Used to load a list of images for pre-processing
    def load(self, image_paths, verbose=-1):
        """
        :param image_paths: List of image paths
        :param verbose: Parameter for printing information to console
        :return: Tuple of data and labels
        """
        data, labels = [], []

        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)

            data.append(image)
            labels.append(label)

            if verbose > 0 and i > 0 and (i+1) % verbose == 0:
                print('[INFO]: Processed {}/{}'.format(i+1, len(image_paths)))

        return (np.array(data), np.array(labels))

In [5]:
#Class Preprocessror 
class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)

In [6]:
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from __main__ import SimplePreprocessor
from __main__ import SimpleDatasetLoader

In [7]:
# Function to load and preprocess data using SimpleDatasetLoader
def load_and_preprocess_data(image_paths, target_size):
    sp = SimplePreprocessor(target_size[0], target_size[1])
    sdl = SimpleDatasetLoader(preprocessors=[sp])
    data, labels = sdl.load(image_paths, verbose=1000)

    print(data.shape)
    print(labels.shape)

    # Convert labels to one-hot encoding
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    #labels = to_categorical(labels)

    return data, labels



# Get list of image paths
image_paths = list(paths.list_images("../LC25000/"))

# Define target size for images
target_size = (224, 224)  # Change this to your desired size

# Load and preprocess data
print('[INFO]: Images loading....')
data, labels = load_and_preprocess_data(image_paths, target_size)

[INFO]: Images loading....
[INFO]: Processed 1000/25000
[INFO]: Processed 2000/25000
[INFO]: Processed 3000/25000
[INFO]: Processed 4000/25000
[INFO]: Processed 5000/25000
[INFO]: Processed 6000/25000
[INFO]: Processed 7000/25000
[INFO]: Processed 8000/25000
[INFO]: Processed 9000/25000
[INFO]: Processed 10000/25000
[INFO]: Processed 11000/25000
[INFO]: Processed 12000/25000
[INFO]: Processed 13000/25000
[INFO]: Processed 14000/25000
[INFO]: Processed 15000/25000
[INFO]: Processed 16000/25000
[INFO]: Processed 17000/25000
[INFO]: Processed 18000/25000
[INFO]: Processed 19000/25000
[INFO]: Processed 20000/25000
[INFO]: Processed 21000/25000
[INFO]: Processed 22000/25000
[INFO]: Processed 23000/25000
[INFO]: Processed 24000/25000
[INFO]: Processed 25000/25000
(25000, 224, 224, 3)
(25000,)


In [8]:
from tensorflow.keras.applications import  ResNet50, EfficientNetB0
from tensorflow.keras.models import Model
with tf.device('/CPU:0'):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(target_size[0], target_size[1], 3))
    model = Model(inputs=base_model.input, outputs=base_model.output)
    features = model.predict(data, batch_size=32, verbose=1)

# Flatten the features
features_flatten = features.reshape(features.shape[0], -1)
    
print(features_flatten.shape)

2024-04-20 18:04:34.426275: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1288 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:65:00.0, compute capability: 8.0


782/782 [==============================] - 1572s 2s/step
(25000, 100352)


In [9]:
data = features_flatten

In [10]:
num_classes = len(np.unique(labels))

In [11]:
import gc
import os

def calculate_sensitivity_specificity(y_true, y_pred, class_label):
    # Create a binary confusion matrix for the specified class
    true_positive = sum((y_true == class_label) & (y_pred == class_label))
    false_positive = sum((y_true != class_label) & (y_pred == class_label))
    true_negative = sum((y_true != class_label) & (y_pred != class_label))
    false_negative = sum((y_true == class_label) & (y_pred != class_label))

    # Calculate sensitivity and specificity
    sensitivity = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    specificity = true_negative / (true_negative + false_positive) if (true_negative + false_positive) > 0 else 0

    return sensitivity, specificity
# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=['Model', 'Average Accuracy', 'Average Sensitivity', 'Average Specificity',
                                   'Average AUC-ROC', 'Average MCC', 'Average Precision', 'Average F1 Score',
                                   'Memory Used (MB)', 'Time (s)'])

# Function to evaluate a model
def evaluate_model(model, name, data, labels):
    accuracy_list = []
    sensitivity_list = []
    specificity_list = []
    auc_roc_list = []
    mcc_list = []
    precision_list = []
    f1_list = []
    time_start = time.time()
    memory_start = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    if name == "VGG16" or name == "EfficientNetB0" or name == "ResNet50":
        with tf.device('/GPU:0'):
            tf.random.set_seed(0)
            for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
                print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

                X_train, X_test = data[train_index], data[test_index]
                y_train, y_test = to_categorical(labels[train_index]), to_categorical(labels[test_index])

                # Adding early stopping to prevent overfitting
                early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
                keras.utils.set_random_seed(0)
                np.random.seed(0)
                tf.random.set_seed(0)
                #tf.config.run_functions_eagerly(True)
                tf.data.experimental.enable_debug_mode()
                # Train the model
                # Example of using tf.data.Dataset
                #train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                #train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                model.fit(train_dataset, epochs=20, callbacks=[early_stopping])


                # Evaluate the model
                predictions = model.predict(X_test)
                y_pred = np.argmax(predictions, axis=1)
                y_test_encoded = np.argmax(y_test, axis=1)

                # print(y_test)
                # print(y_test_encoded)
                # print(y_pred)
                # print(predictions)
                # Calculate evaluation metrics for the current fold
                # Calculate evaluation metrics for the current fold
                accuracy = np.mean(y_pred == y_test_encoded)
                sensitivities = []
                specificities = []
                for class_label in range(num_classes):
                    sen, spe = calculate_sensitivity_specificity(y_test_encoded, y_pred, class_label)
                    sensitivities.append(sen)
                    specificities.append(spe)
                sensitivity = sum(sensitivities) / num_classes
                specificity = sum(specificities) / num_classes
                auc_roc = roc_auc_score(y_test, predictions,multi_class='ovr')
                mcc = matthews_corrcoef(y_test_encoded, y_pred)
                precision = precision_score(y_test_encoded, y_pred, average='macro')
                f1 = f1_score(y_test_encoded, y_pred, average='macro')
                # Append metrics to lists
                accuracy_list.append(accuracy)
                sensitivity_list.append(sensitivity)
                specificity_list.append(specificity)
                auc_roc_list.append(auc_roc)
                mcc_list.append(mcc)
                precision_list.append(precision)
                f1_list.append(f1)

                # Print metrics for the current fold
                print(f"Accuracy: {accuracy}")
                print(f"Sensitivity: {sensitivity}")
                print(f"Specificity: {specificity}")
                print(f"AUC-ROC: {auc_roc}")
                print(f"MCC: {mcc}")
                print(f"Precision: {precision}")
                print(f"F1 Score: {f1}")
                gc.collect()

            # Calculate average metrics
            average_accuracy = np.mean(accuracy_list)
            average_sensitivity = np.mean(sensitivity_list)
            average_specificity = np.mean(specificity_list)
            average_auc_roc = np.mean(auc_roc_list)
            average_mcc = np.mean(mcc_list)
            average_precision = np.mean(precision_list)
            average_f1 = np.mean(f1_list)

            time_end = time.time()
            memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
            del X_train, X_test, y_train, y_test
            gc.collect()
            tf.keras.backend.clear_session()
    else:
        for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
            print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

            X_train, X_test = data[train_index], data[test_index]
            y_train, y_test = labels[train_index], labels[test_index]

            # Train the model
            model.fit(X_train, y_train)

            # Evaluate the model
            predictions = model.predict(X_test)
            y_prob = model.predict_proba(X_test)
            y_pred = predictions
            y_test_encoded = y_test

            # Calculate evaluation metrics for the current fold
            accuracy = np.mean(y_pred == y_test_encoded)
            sensitivities = []
            specificities = []
            for class_label in range(num_classes):
                sen, spe = calculate_sensitivity_specificity(y_test, y_pred, class_label)
                sensitivities.append(sen)
                specificities.append(spe)
            sensitivity = sum(sensitivities) / num_classes
            specificity = sum(specificities) / num_classes
            auc_roc = roc_auc_score(y_test, y_prob,multi_class='ovr')
            mcc = matthews_corrcoef(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='macro')
            f1 = f1_score(y_test, y_pred, average='macro')
            # Append metrics to lists
            accuracy_list.append(accuracy)
            sensitivity_list.append(sensitivity)
            specificity_list.append(specificity)
            auc_roc_list.append(auc_roc)
            mcc_list.append(mcc)
            precision_list.append(precision)
            f1_list.append(f1)

            # Print metrics for the current fold
            print(f"Accuracy: {accuracy}")
            print(f"Sensitivity: {sensitivity}")
            print(f"Specificity: {specificity}")
            print(f"AUC-ROC: {auc_roc}")
            print(f"MCC: {mcc}")
            print(f"Precision: {precision}")
            print(f"F1 Score: {f1}")
            gc.collect()

        # Calculate average metrics
        average_accuracy = np.mean(accuracy_list)
        average_sensitivity = np.mean(sensitivity_list)
        average_specificity = np.mean(specificity_list)
        average_auc_roc = np.mean(auc_roc_list)
        average_mcc = np.mean(mcc_list)
        average_precision = np.mean(precision_list)
        average_f1 = np.mean(f1_list)

        time_end = time.time()
        memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
        

    # Append results to DataFrame
    results_df.loc[len(results_df)] = [name, average_accuracy, average_sensitivity, average_specificity,
                                       average_auc_roc, average_mcc, average_precision, average_f1,
                                       memory_end - memory_start, time_end - time_start]





In [12]:
from sklearn.linear_model import LogisticRegression
# SVM
lr_model = LogisticRegression(n_jobs=-1, random_state=0)
evaluate_model(lr_model, 'LR', data, labels)




[INFO] Fold 1 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9964
Sensitivity: 0.9964000000000001
Specificity: 0.9991
AUC-ROC: 0.9999564
MCC: 0.9955009955014932
Precision: 0.9964039472535753
F1 Score: 0.9963999867999483

[INFO] Fold 2 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.996
Sensitivity: 0.9960000000000001
Specificity: 0.999
AUC-ROC: 0.9999820000000001
MCC: 0.9950031840152833
Precision: 0.9960126472094213
F1 Score: 0.9959999615993856

[INFO] Fold 3 / 10 for LR


...

Accuracy: 0.9928
Sensitivity: 0.9927999999999999
Specificity: 0.9982
AUC-ROC: 0.9998576
MCC: 0.991004162226222
Precision: 0.9928114766138719
F1 Score: 0.9927974570025462

[INFO] Fold 9 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9952
Sensitivity: 0.9952
Specificity: 0.9987999999999999
AUC-ROC: 0.9998962
MCC: 0.9940005964005367
Precision: 0.9952063553401624
F1 Score: 0.995201990013966

[INFO] Fold 10 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9956
Sensitivity: 0.9955999999999999
Specificity: 0.9989000000000001
AUC-ROC: 0.9999488000000001
MCC: 0.9945073593816892
Precision: 0.995629026468035
F1 Score: 0.9955998555948155


In [13]:
#NB
nb_model = GaussianNB()
evaluate_model(nb_model, 'NB', data, labels)


[INFO] Fold 1 / 10 for NB
Accuracy: 0.8824
Sensitivity: 0.8824
Specificity: 0.9705999999999999
AUC-ROC: 0.9270291
MCC: 0.8540276719895162
Precision: 0.8864489662393715
F1 Score: 0.8822311606344634

[INFO] Fold 2 / 10 for NB
Accuracy: 0.8624
Sensitivity: 0.8624
Specificity: 0.9655999999999999
AUC-ROC: 0.9153164
MCC: 0.8285777200765543
Precision: 0.8637431580202938
F1 Score: 0.861820964891356

[INFO] Fold 3 / 10 for NB
Accuracy: 0.8648
Sensitivity: 0.8648
Specificity: 0.9662
AUC-ROC: 0.9164528000000001
MCC: 0.8314772385985666
Precision: 0.8662727013251658
F1 Score: 0.8645218258001857

[INFO] Fold 4 / 10 for NB
Accuracy: 0.8544
Sensitivity: 0.8544
Specificity: 0.9636000000000001
AUC-ROC: 0.9106603
MCC: 0.8183834206123626
Precision: 0.8560149355778417
F1 Score: 0.8544226895510256

[INFO] Fold 5 / 10 for NB
Accuracy: 0.8664
Sensitivity: 0.8664000000000002
Specificity: 0.9666
AUC-ROC: 0.9164273999999999
MCC: 0.8338522990857256
Precision: 0.8703391069142749
F1 Score: 0.866559740481815

[INFO

In [14]:

# KNN
knn_model = KNeighborsClassifier()
evaluate_model(knn_model, 'KNN', data, labels)




[INFO] Fold 1 / 10 for KNN
Accuracy: 0.9656
Sensitivity: 0.9656
Specificity: 0.9914
AUC-ROC: 0.9965645999999999
MCC: 0.9575507500674822
Precision: 0.967641371496927
F1 Score: 0.9655369051776285

[INFO] Fold 3 / 10 for KNN
Accuracy: 0.954
Sensitivity: 0.954
Specificity: 0.9884999999999999
AUC-ROC: 0.9971588
MCC: 0.9434909972967699
Precision: 0.9576027164607073
F1 Score: 0.9538937377116318

[INFO] Fold 4 / 10 for KNN
Accuracy: 0.9584
Sensitivity: 0.9583999999999999
Specificity: 0.9896
AUC-ROC: 0.9959025
MCC: 0.9488110107394997
Precision: 0.9613797047027299
F1 Score: 0.9583263548629976

[INFO] Fold 5 / 10 for KNN
Accuracy: 0.9548
Sensitivity: 0.9548
Specificity: 0.9887
AUC-ROC: 0.9972919000000001
MCC: 0.9444322369445334
Precision: 0.9583122759202034
F1 Score: 0.9547579708012656

[INFO] Fold 6 / 10 for KNN
Accuracy: 0.9652
Sensitivity: 0.9652
Specificity: 0.9913000000000001
AUC-ROC: 0.9971288000000001
MCC: 0.9569925951668375
Precision: 0.9670941627044094
F1 Score: 0.9651919777235177

[INF

In [15]:
# Random Forest
rf_model = RandomForestClassifier(random_state=0, n_jobs=-1)
evaluate_model(rf_model, 'Random Forest', data, labels)



[INFO] Fold 1 / 10 for Random Forest
Accuracy: 0.9728
Sensitivity: 0.9728
Specificity: 0.9931999999999999
AUC-ROC: 0.9982599000000001
MCC: 0.9660276287852805
Precision: 0.9728006508061873
F1 Score: 0.9727450426108412

[INFO] Fold 2 / 10 for Random Forest
Accuracy: 0.972
Sensitivity: 0.9720000000000001
Specificity: 0.993
AUC-ROC: 0.9985373
MCC: 0.9650088781225183
Precision: 0.971968718437533
F1 Score: 0.971966531244959

[INFO] Fold 3 / 10 for Random Forest
Accuracy: 0.9628
Sensitivity: 0.9628
Specificity: 0.9907
AUC-ROC: 0.9965126000000002
MCC: 0.953517163463415
Precision: 0.9627741212882114
F1 Score: 0.962753146124878

[INFO] Fold 4 / 10 for Random Forest
Accuracy: 0.9728
Sensitivity: 0.9728
Specificity: 0.9932000000000001
AUC-ROC: 0.9981674999999999
MCC: 0.9660137174921832
Precision: 0.9728491806423001
F1 Score: 0.9727968465437117

[INFO] Fold 5 / 10 for Random Forest
Accuracy: 0.9664
Sensitivity: 0.9663999999999999
Specificity: 0.9916
AUC-ROC: 0.9985182
MCC: 0.9580051732419033
Preci

In [ ]:
import lightgbm
# LGBM
lgb_model = lightgbm.LGBMClassifier(random_state=0, n_jobs = -1, force_col_wise = True)
evaluate_model(lgb_model, 'LGBM', data, labels)


[INFO] Fold 1 / 10 for LGBM
[LightGBM] [Info] Total Bins 21363478
[LightGBM] [Info] Number of data points in the train set: 22500, number of used features: 99704
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
Accuracy: 0.9888
Sensitivity: 0.9888
Specificity: 0.9972000000000001
AUC-ROC: 0.9998168
MCC: 0.9860017748047919
Precision: 0.9888102866124886
F1 Score: 0.9888015692038442

[INFO] Fold 2 / 10 for LGBM
[LightGBM] [Info] Total Bins 21367897
[LightGBM] [Info] Number of data points in the train set: 22500, number of used features: 99714
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info]

In [ ]:
# Save results to a CSV file
results_df.to_csv('DS6_ResNet_LGBM.csv', index=False)

In [ ]:
results_df